In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set001'

In [3]:
#gdf = combine_merged_data(setID = set_ID)

In [47]:
gdf = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{set_ID}.shp'), driver='ESRI Shapefile')

In [5]:
# this will take about 17 minutes
all_taxlot = combine_taxlot()

In [20]:
n_gdf = merge_matched(setID = set_ID, all_taxlot = all_taxlot)

it is about 8.38% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [25]:
n_gdf['lots'] = n_gdf['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [38]:
n_gdf1 = n_gdf[['record_ID', 'wdID', 'parcel_id', 'ORTaxlot', 'notes', 'geometry']]

In [39]:
n_gdf1.to_file(os.path.join(inpath + f'\\{outfolder}\\', f'combined_records_in_{set_ID}.shp'), 
                                                  driver='ESRI Shapefile')

In [41]:
n_gdf1.notes.unique()

array(['partial', None, 'ROW & partial', 'ROW'], dtype=object)

In [36]:
wd_df = combine_wd_table(set_ID)

In [40]:
# records to review and map
wd_df.shape[0] - len(n_gdf1.record_ID.unique())

112

In [44]:
# to check the parcel IDs with notes?
len(n_gdf1.record_ID.unique()) - len(n_gdf1[n_gdf1.notes.isnull()].record_ID.unique())

418

In [48]:
df_wo_lots = records_without_lots(gdf = gdf, setID = set_ID)

it is about 8.38% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


In [50]:
df_wo_lots.shape[0]

61

In [51]:
setgdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_{set_ID}_Combined')

In [59]:
to_map_rID = [rID for rID in wd_df.record_ID.values if rID not in n_gdf1.record_ID.unique()]

In [67]:
mapped = setgdf[setgdf.Record_ID.isin(to_map_rID)]

In [68]:
mapped.rename(columns={'wetdet_delin_number': 'wdID', 
                      'address_location_desc':'loc_desc', 
                      'Coord-Source': 'coord_src',
                      'DocumentName':'doc_name',
                      'DecisionLink':'doc_link',
                      'is_batch_file':'isbatfile',
                      'status_name': 'status_nm',
                      'received_date':'receiveddt', 
                      'response_date':'responsedt',
                      'reissuance_response_date':'reissuance', 
                      'Match_found':'matchfound', 
                      'Manual_note': 'notes',
                      'Edits_Complete': 'edits'
                      }, inplace=True)

In [81]:
selcols = list(mapped.columns[list(map(lambda x: x <= 10, list(map(len, list(mapped.columns)))))])

In [82]:
mapped[selcols].to_file(os.path.join(inpath + f'\\{outfolder}\\', f'mapped_in_{set_ID}.shp'), 
                                                  driver='ESRI Shapefile')